In [1]:
from hw2skeleton import cluster
from hw2skeleton import io
import os
import numpy as np
from random import randint
import random
from statistics import mean 
from scipy.stats.mstats import gmean
import pandas as pd
import math

In [ ]:
#####################overall framework is distilling the data into simpler pieces in stages
#####################in the end the reduced information is used to calculate similarity

###########First calculate for each residue two pieces of information:
####1) centroid of residue (simplifies all the atomic positions of side chains etc)
####2) class of amino acid (ie polar amphipathic etc)
###########then reduce the information from residues to properties of active site:
####1) most frequent amino acid class
####2) ratio of first longest dimension / second longest dimension
####3) ratio of first longest dimension / third longest dimension
###########then combine these three properties into a similarity score with each one being weighted 

def calc_cent(residue):
    ######pull out only the coordinates from the residue and calculate the centroid of the residue
    tmp_activesite = []
    for atom in residue.atoms:
        tmp_atm = []
        for coord in atom.coords:
            tmp_atm.append(coord)
        tmp_activesite.append(tmp_atm)
    tmp_activesite = np.array(tmp_activesite)
    ######calc centroid in 3d
    length = tmp_activesite.shape[0]
    sum_x = np.sum(tmp_activesite[:, 0])
    sum_y = np.sum(tmp_activesite[:, 1])
    sum_z = np.sum(tmp_activesite[:, 2])
    return sum_x/length, sum_y/length, sum_z/length

def calc_class(residue):
    #####classify residue by type of amino acid class it falls into
    classdict = {'ARG': 'charged', 
         'LYS': 'charged', 
         'ASP': 'charged', 
         'GLU': 'charged', 
         'GLN': 'polar',
         'ASN': 'polar', 
         'HIS': 'polar', 
         'SER': 'polar', 
         'THR': 'polar', 
         'TYR': 'polar', 
         'CYS': 'polar', 
         'TRP': 'amphipathic', 
         'TYR': 'amphipathic', 
         'MET': 'amphipathic', 
         'ALA': 'hydrophobic', 
         'ILE': 'hydrophobic', 
         'LEU': 'hydrophobic', 
         'PHE': 'hydrophobic', 
         'VAL': 'hydrophobic',
         'PRO': 'hydrophobic',
         'GLY': 'hydrophobic'}
    ######check if labelled correctly
    if residue.type in classdict:
        ######if so return the class 
        return classdict.get(residue.type)
    else:
        return 'NA'

###########
def calc_ratios(cent):
    x_vals = []
    y_vals = []
    z_vals = []
    ####pull out values individually
    for residue in cent:
        x_vals.append(residue[0])
        y_vals.append(residue[1])
        z_vals.append(residue[2])
    ####calc distance in each dimension
    dists = []
    dists.append(max(x_vals)-min(x_vals)),
    dists.append(max(y_vals)-min(y_vals)),
    dists.append(max(z_vals)-min(z_vals))
    #####calc ratios of distances
    dists.sort()
    ratio1to2 = dists[2]/dists[0]
    ratio1to3 = dists[2]/dists[1]
    return ratio1to2, ratio1to3

def calc_mostcommon(lst):
    return max(set(lst), key=lst.count)


In [ ]:
filename_a = os.path.join("data", "10701.pdb")
filename_b = os.path.join("data", "4629.pdb")

activesite_a = io.read_active_site(filename_a)
activesite_b = io.read_active_site(filename_b)

#########combine into one
activesites = []
activesites.append(activesite_a)
activesites.append(activesite_b)
########generate properties of activesites
props = []
for site in activesites:
    class_res = []
    cent_res = []
    ####calculate centroids and classes of residues
    for residue in site.residues:
        class_res.append(calc_class(residue))
        cent_res.append(calc_cent(residue))
    ####calculate active site properties
    ###0 = ratio1to2
    ###1 = ratio1to3
    ratio1to2, ratio1to3 = calc_ratios(cent_res)
    common = calc_mostcommon(class_res)
    props.append((ratio1to2,ratio1to3,common))
###########compute similarity across activesites
diff1 = abs((props[0][0]-props[1][0])/props[0][0])
diff2 = abs((props[0][1]-props[1][1])/props[0][1])
if diff1 >=1:
    diff1 = 1
if diff2 >=1:
    diff2 = 1
if props[0][2] == props[1][2]:
    diff3 = 1
else:
    diff3 = 0
# print(props)
# print(diff1,diff2,diff3)
########combine into one similarity metric
similarity = (1/3)*(1-diff1)+(1/3)*(1-diff2)+(1/3)*diff3
print(similarity)
print(diff1,diff2,diff3)

In [ ]:
filename_a = os.path.join("data", "10701.pdb")
filename_b = os.path.join("data", "4629.pdb")

site_a = io.read_active_site(filename_a)
site_b = io.read_active_site(filename_b)

cluster.compute_similarity(site_a,site_b)



In [ ]:
class_b = []
cent_b = []
for 
for residue in activesite_b.residues:
    class_b.append(calc_class(residue))
    cent_b.append(calc_cent(residue))
    
cent_b
    

In [ ]:
def find_closest(centroid_list,test):
    closest_vector = []
    sim_matrix = []
    #####calculate distances from centroids
    for site in test:
        sim_vector = []
        for centroid in centroid_list:
            sim = cluster.compute_similarity(centroid,site)
            sim_vector.append(sim)
        sim_matrix.append(sim_vector)
        #####find closest
        closest = sim_vector.index(max(sim_vector))
        closest_vector.append(closest)
    return(closest_vector)

def find_newcentroid(closest_vector,test,n_clusters):
    ####split up test vector by cluster identity
    clusters = []
    for i in range(n_clusters):
        clusters.append([])
    i = 0
    for centroid in closest_vector:
        tmp = test[i]
        clusters[centroid].append(tmp)
        i+=1
    #####okay so now that they are split up by cluster find the point with smallest intracluster distance
    new_centroids = []
    ##this is for intracluster distance
    new_dists = []
    for k in clusters:
        dists = []
        for site_r in k:
            ###calc intracluster distance
            ##row contains intracluster distances
            row = []
            for site_c in k:
                sim = cluster.compute_similarity(site_r,site_c)
                row.append(sim)
            row_sum = sum(row)
            ####this is sum of final distances
            dists.append(row_sum)
        #####find point with smallest distance; this is the new centroid
        new_cent = dists.index(min(dists))
        new_centroids.append(k[new_cent])
        #####calculate total distance as well to measure cluster 'goodness'
        new_dists.append(sum(dists))
        #####
    return(new_centroids,new_dists)




In [ ]:
############################K-means implementation
####import data
active_sites = io.read_active_sites('/Users/johnny/Desktop/class/hw2-skeleton/data')
test = active_sites[0:10]
####define k 
n_clusters = 4
###Randomize cluster centroids
rnd_nums = random.sample(range(0, len(test)-1), n_clusters)
centroids = []
for num in rnd_nums:
    centroids.append(test[num])
###Main
i = 0
while i < 50:
    ###return vector indicating labels
    labels = find_closest(centroids,test)
    ###Find New Centroids
    new_centroids, new_dists = find_newcentroid(labels,test,n_clusters)
    ###check if final centers have been reached
    if np.all(centroids == new_centroids):
        break
    ###update centroids
    centroids = new_centroids
    i+=1
    ###
    print(i)
    print('new_dists:',new_dists)
    print('new_dists:',gmean(new_dists))

In [ ]:
#############Hierarchical Implementation
###########K-nearest Neighbor
test = active_sites[0:10]
####
def getNeighbors(testSet, k):
    distances = []
    closest_neighbors = []
    for item1 in testSet:
        ####get the distance of this point to each other point
        item1_distances = []
        item1_ids = []
        for item2 in testSet:
            ####don't calculate self
            if item2 != item1:
                dist = cluster.compute_similarity(item1,item2)
                item1_distances.append(dist)
                item1_ids.append(item2)
        ####find the closest neighbor NOT in the original cluster
        neighbor = item1_ids[item1_distances.index(min(item1_distances))]
        closest_neighbors.append(neighbor)
    return(closest_neighbors)

def calc_distmatrix(testSet):
    DistMatrix = []
    for item1 in testSet:
        row = []
        for item2 in testSet:
            dist = cluster.compute_similarity(item1,item2)
            row.append(dist)
        DistMatrix.append(row)
    return DistMatrix


neighbors = getNeighbors(test,2)
print(neighbors)
print(test)

In [2]:
def calc_distmatrix(testSet):
    DistMatrix = []
    for item1 in testSet:
        row = []
        for item2 in testSet:
            dist = cluster.compute_similarity(item1,item2)
            row.append(dist)
        DistMatrix.append(row)
    return DistMatrix


In [48]:
######data structures
####test contains the active sites in an array
####clust contains the cluster membership in an array
####
active_sites = io.read_active_sites('/Users/johnny/Desktop/class/hw2-skeleton/data')

test = active_sites[0:10]
def agglomerative(test):
    #######this is agglomerative
    ######this is an array of memberships which is returned
    clust_all = []
    ######initialize clust, each point is its own cluster to start
    clust = []
    i = 0
    for item in test:
        clust.append(i)
        i+=1
    #####final output will be an array of clusters

    ######calculate distance matrix
    DistMatrix = calc_distmatrix(test)
    ######Begin agglomeration
    while True:
        closest_clust = []
        for clust_id in clust:
            #####define self and other because we want to find closest neighbor not in same cluster
            self_clust = []
            other_clust = []
            #####iterate through test to assign self and other clusters; appending index for easy access
            for i in range(len(test)):
                if clust[i] == clust_id:
                    self_clust.append(i)
                else:
                    other_clust.append(i)
            #####now use distance matrix to find those points closest to each cluster (find closest neighbors)
            ##min_val holds current minimum, ind_self = index of that item in self, ind_other = index of that item in other
            min_val = -1 ##similarity is 0<sim<1
            ind_self = math.inf
            ind_other = math.inf
            for row in range(len(DistMatrix)):
                ####compare distances between those in and those out of self cluster
                if row in self_clust:
                    for col in range(len(DistMatrix)):
                        if col not in self_clust:
                            ###compare max sim
                            if DistMatrix[row][col] > min_val:
                                ##update
                                min_val = DistMatrix[row][col]
                                ind_self = row
                                ind_other = col
            ####remember which cluster was the closest; store the cluster ID!!
            closest_clust.append(clust[ind_other])
        #####go through clusters and combine clusters based on closest neighbors
        ####neighbors have to be reciprocal; this is written out long form for my own clarity
        for i in range(len(closest_clust)):
            self_index = i
            neighbor_index = closest_clust[i] ##index of neighbor is stored here
            self_neighbor = closest_clust[i] ##index of neighbor is stored here
            neighbor_neighbor = closest_clust[closest_clust[i]] 
            if self_index == neighbor_neighbor:
                ###update clusters
                self_cluster_id = clust[self_index]
                neighbor_cluster_id = clust[neighbor_index]
                ##find neighbor clusters and update to self cluster id
                for j in range(len(clust)):
                    if (clust[j] == neighbor_cluster_id):
                        clust[j] = self_cluster_id
        #####save cluster membership and number of clusters
        clust_all.append(clust)
        #####see if num sets == 1 yet
        if len(set(clust)) == 1:
            break
                        


Read in 136 active sites
0 7 [7, 7, 6, 9, 5, 4, 4, 1, 4, 3] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
1 7 [7, 7, 6, 9, 5, 4, 4, 1, 4, 3] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2 6 [7, 7, 6, 9, 5, 4, 4, 1, 4, 3] [0, 1, 2, 3, 4, 5, 6, 1, 8, 9]
3 9 [7, 7, 6, 9, 5, 4, 4, 1, 4, 3] [0, 1, 2, 3, 4, 5, 6, 1, 8, 9]
4 5 [7, 7, 6, 9, 5, 4, 4, 1, 4, 3] [0, 1, 2, 3, 4, 5, 6, 1, 8, 3]
5 4 [7, 7, 6, 9, 5, 4, 4, 1, 4, 3] [0, 1, 2, 3, 4, 4, 6, 1, 8, 3]
6 4 [7, 7, 6, 9, 5, 4, 4, 1, 4, 3] [0, 1, 2, 3, 4, 4, 6, 1, 8, 3]
7 1 [7, 7, 6, 9, 5, 4, 4, 1, 4, 3] [0, 1, 2, 3, 4, 4, 6, 1, 8, 3]
8 4 [7, 7, 6, 9, 5, 4, 4, 1, 4, 3] [0, 1, 2, 3, 4, 4, 6, 1, 8, 3]
9 3 [7, 7, 6, 9, 5, 4, 4, 1, 4, 3] [0, 1, 2, 3, 4, 4, 6, 1, 8, 3]
[0, 1, 2, 3, 4, 4, 6, 1, 8, 3]
0 1 [1, 0, 6, 4, 8, 8, 4, 0, 4, 4] [0, 1, 2, 3, 4, 4, 6, 1, 8, 3]
1 0 [1, 0, 6, 4, 8, 8, 4, 0, 4, 4] [0, 0, 2, 3, 4, 4, 6, 0, 8, 3]
2 6 [1, 0, 6, 4, 8, 8, 4, 0, 4, 4] [0, 0, 2, 3, 4, 4, 6, 0, 8, 3]
3 4 [1, 0, 6, 4, 8, 8, 4, 0, 4, 4] [0, 0, 2, 3, 4, 4, 6, 0, 8, 3]
4 8 [1, 0, 6, 4, 8, 

In [10]:
DistMatrix

[[1.0, 0.91214520996876347, 0.36403547508989237, 0.54501586304459726],
 [0.89313403416281467, 1.0, 0.3333333333333333, 0.41346003763254058],
 [0.61057433475226275, 0.57940750581593081, 1.0, 0.31218521860040938],
 [0.57614799227375313, 0.51216513190659252, 0.024976586830300018, 1.0]]

In [ ]:
for num in set(clust):
    print(num)

In [ ]:
def simple_hierarchical_clustering(distance_matrix, howmany):
    site_names = distance_matrix.index.tolist()
    clusters_number = distance_matrix.shape[0]
    cldict = {x:set([x]) for x in site_names}
    pairs_dict = {}
    # get all the distances between pairs of points in convenient form
    for p in site_names:
    for q in site_names:
    if p == q:
    continue
    pairs_dict[(p,q)] = distance_matrix.ix[p][q]

    # sort the pairs by distance
    sorted_pairs_dict = sorted(pairs_dict.items(), key=operator.itemgetter(1))
    # iteratively combine them to clusters
    for element in sorted_pairs_dict:
    pair = element[0]
    value = element[1]
    if cldict[pair[0]] != cldict[pair[1]]:
    clusters_number -= 1
    cldict[pair[0]] = (cldict[pair[0]] | cldict[pair[1]])
    for el in cldict[pair[0]]:
    cldict[el] = cldict[pair[0]]
    if clusters_number <= howmany:
    break

    clusters_set = set([frozenset(cldict[k]) for k in cldict])
    return clusters_set

In [ ]:
def simple_hierarchical_clustering(distance_matrix, howmany):
    site_names = distance_matrix.index.tolist()
    clusters_number = distance_matrix.shape[0]
    cldict = {x:set([x]) for x in site_names}
    pairs_dict = {}
    # get all the distances between pairs of points in convenient form
    for p in site_names:
        for q in site_names:
            if p == q:
                continue
    pairs_dict[(p,q)] = distance_matrix.ix[p][q]

    # sort the pairs by distance
    sorted_pairs_dict = sorted(pairs_dict.items(), key=operator.itemgetter(1))
    # iteratively combine them to clusters
    for element in sorted_pairs_dict:
    pair = element[0]
    value = element[1]
    if cldict[pair[0]] != cldict[pair[1]]:
    clusters_number -= 1
    cldict[pair[0]] = (cldict[pair[0]] | cldict[pair[1]])
    for el in cldict[pair[0]]:
    cldict[el] = cldict[pair[0]]
    if clusters_number <= howmany:
    break

    clusters_set = set([frozenset(cldict[k]) for k in cldict])
    return clusters_set

In [ ]:
def Classify(nItem, k, Items): 
    if(k > len(Items)): 
          
        # k is larger than list 
        # length, abort 
        return "k larger than list length"; 
      
    # Hold nearest neighbors. 
    # First item is distance,  
    # second class 
    neighbors = []; 
  
    for item in Items: 
        
        # Find Euclidean Distance 
        distance = EuclideanDistance(nItem, item); 
  
        # Update neighbors, either adding 
        # the current item in neighbors  
        # or not. 
        neighbors = UpdateNeighbors(neighbors, item, distance, k); 
  
    # Count the number of each 
    # class in neighbors 
    count = CalculateNeighborsClass(neighbors, k); 
  
    # Find the max in count, aka the 
    # class with the most appearances. 
    return FindMax(count); 

In [ ]:
print(sim_matrix)

In [ ]:
test = [1,2,0]

print(test.index(min(test)))

In [ ]:
Step 1 - Pick K random points as cluster centers called centroids.
Step 2 - Assign each x_ix 
i
​	  to nearest cluster by calculating its distance to each centroid.
Step 3 - Find new cluster center by taking the average of the assigned points.
Step 4 - Repeat Step 2 and 3 until none of the cluster assignments change.

In [ ]:
x_vals = []
y_vals = []
z_vals = []
####pull out values individually
for residue in cent_b:
    x_vals.append(residue[0])
    y_vals.append(residue[1])
    z_vals.append(residue[2])
####calc distance in each dimension
dists = []
dists.append(max(x_vals)-min(x_vals)),
dists.append(max(y_vals)-min(y_vals)),
dists.append(max(z_vals)-min(z_vals))
print(dists)
#####calc ratios of distances
dists = dists.sort()
ratio1to2 = dists[2]/dists[0]
ratio1to3 = dists[2]/dists[1]
print(ratio1to2)
print(ratio1to3)




In [ ]:
1.70394823498
1.43196483847

In [ ]:
cent_b[:,0]

In [ ]:
dists = distance.cdist(cent_b, cent_b, 'euclidean')
for dist in dists:
    print(dist)
    print('okay')

In [ ]:
activesite_a.residues

In [ ]:
activesite_b.residues

In [ ]:
activesite_b.residues[0].atoms[0].coords

In [ ]:
activesite_b.residues[0].atoms[2].coords

In [ ]:
activesite_b.residues[0].atoms

In [ ]:
activesite_b.residues[1].atoms[0].coords

tmp_activesite = []
for atom in activesite_b.residues[1].atoms:
    tmp_atm = []
    for coord in atom.coords:
        tmp_atm.append(coord)
    tmp_activesite.append(tmp_atm)
tmp_activesite = np.array(tmp_activesite)
tmp_activesite

In [ ]:
calc_centroid(activesite_b.residues[1])

In [ ]:
for residue in activesite_b.residues:
    print(calc_class(residue))

In [ ]:
activesite_b.residues[1].type

In [ ]:
activesite_b.residues